In [2]:
import utils
import numpy as np

image_files, labels = utils.getprocessedfilelist()

In [2]:
np.save('data_road\\images.npy', image_files)
np.save('data_road\\labels.npy', labels)

In [3]:
import utils
import numpy as np

image_files = np.load('data_road\\images.npy')
labels = np.load('data_road\\labels.npy')

In [4]:
import tensorflow as tf
data = tf.data.Dataset.from_tensor_slices((image_files, labels))
a = data.as_numpy_iterator()
b = a.next()
print(b)

(b'data_road\\processed_images\\umm_000000_0.jpg', array([160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,
       160., 160., 160., 160., 160., 160., 160., 160., 160., 160., 160.,


In [6]:
import numpy as np
import tensorflow as tf

@tf.function
def readimage(image_file, label):
    image = tf.io.decode_jpeg(tf.io.read_file(image_file)) / 255
    return image, label

shuffled_ds = data.shuffle(20000).map(readimage, num_parallel_calls=tf.data.AUTOTUNE)

iter_ = shuffled_ds.as_numpy_iterator()
a = iter_.next()
print(a)

(array([[[0.11764706, 0.15686275, 0.12156863],
        [0.09019608, 0.11764706, 0.08627451],
        [0.13333334, 0.12941177, 0.10980392],
        ...,
        [0.16078432, 0.23137255, 0.13725491],
        [0.19607843, 0.25882354, 0.15686275],
        [0.23137255, 0.28627452, 0.18431373]],

       [[0.09803922, 0.14509805, 0.10588235],
        [0.07843138, 0.11764706, 0.08235294],
        [0.14901961, 0.16078432, 0.13333334],
        ...,
        [0.10980392, 0.16862746, 0.09411765],
        [0.14117648, 0.2       , 0.11764706],
        [0.18431373, 0.23921569, 0.14901961]],

       [[0.09019608, 0.16078432, 0.11372549],
        [0.07450981, 0.13725491, 0.09411765],
        [0.14117648, 0.18039216, 0.14509805],
        ...,
        [0.03137255, 0.07843138, 0.03921569],
        [0.04313726, 0.09019608, 0.05098039],
        [0.0627451 , 0.11764706, 0.06666667]],

       ...,

       [[0.4862745 , 0.49803922, 0.47058824],
        [0.4627451 , 0.45882353, 0.4392157 ],
        [0.4745098 , 

In [7]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Conv1D, LSTM, MaxPool2D, Flatten, InputLayer, Reshape, TimeDistributed
import tensorflow as tf
import os

def create_model():
    model = Sequential([
        InputLayer(input_shape=(160,600,3)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        MaxPool2D(),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        TimeDistributed(LSTM(64, return_sequences=True)),
        Conv2D(64, (3,3), padding='same', activation='relu'),
        TimeDistributed(LSTM(64, return_sequences=True)),
        Conv2D(64, (5,1), strides=(4,1), activation='relu'),
        Conv2D(1, (4,1), activation='relu'),
        Flatten(),
        Dense(600, activation='linear')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

callbacks = [
    tf.keras.callbacks.TensorBoard(),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join('ckpt','weights.{epoch:02d}-{val_loss:.2f}.hdf5'), 
        monitor='val_loss', mode='min', save_best_only=True)
]

batch_size = 32

train_ds = shuffled_ds.take(20000).batch(batch_size)
val_ds = shuffled_ds.skip(20000).take(5000).batch(batch_size)
model = create_model()


In [8]:
tf.keras.backend.clear_session()

model.fit(x=train_ds, batch_size=batch_size, epochs=80, callbacks=callbacks, validation_data=val_ds)

Epoch 1/80
254/625 [===========>..................] - ETA: 1:28 - loss: 19280.0801 - accuracy: 0.0049